<div style="background-color: #f5f5f5; padding: 15px; color: black; width: 80%;">✍ На практике источники данных редко ограничиваются одной таблицей. Например, если мы работаем с базой данных, то данные в ней могут быть представлены в виде нескольких десятков таблиц, каждая из которых несёт отдельную информацию. Если вы делаете выгрузку из базы напрямую, не объединяя таблицы в единую структуру средствами SQL, вам необходимо знать, как работать с такими таблицами средствами Pandas. </dov>

## С какими данными мы работаем?

В этой части модуля мы будем работать с популярным датасетом <a href="https://grouplens.org/datasets/movielens/">MovieLens</a>, в котором собраны логи некоторой рекомендательной системы фильмов.

Наши данные представляют собой четыре таблицы:
- ratings1 и ratings2 — таблицы с данными о выставленных пользователями оценках фильмов. Они имеют одинаковую структуру и типы данных — на самом деле это две части одной таблицы с оценками фильмов. 

<img src="../static/img/pandas_adv_3.png">

- userId — уникальный идентификатор пользователя, который выставил оценку;
- movieId — уникальный идентификатор фильма;
- rating — рейтинг фильма.


- dates — таблица с датами выставления всех оценок. 

<img src="../static/img/pandas_adv_4.png">

- date — дата и время выставления оценки фильму.

- movies — таблица с информацией о фильмах. 

<img src="../static/img/pandas_adv_5.png">

- movieId — уникальный идентификатор фильма;
- title — название фильма и год его выхода;
- genres — жанры фильма.


Итак, представим, что нам надо получить единую таблицу, в которой будут собраны рейтинги, даты выставления рейтингов, а также информация о фильмах. Вот как мы будем действовать:

<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;"> 
<div style="color: white;background-color: black;">1</div>
Склеим таблицы ratings1 и ratings2 в единую структуру.
<div style="border: 3px dotted white; padding: 5px; margin-right: auto;  width: 80%;"> Термин «склеить» в данном случае обозначает конкатенацию — присоединение одной таблицы к другой.
 </div></div>

<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;"> 
<div style="color: white;background-color: black;">2</div>
К полученной таблице с рейтингами подсоединим столбец с датой проставления рейтинга, склеив столбцы таблиц между собой.
 </div>

<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;"> 
<div style="color: white;background-color: black;">3</div>
Присоединим к нашей таблице информацию о названиях и жанрах фильмов.
 </div>

## Зачем хранить данные в разных таблицах?

Конечно, здорово, если все необходимые данные лежат в одной таблице, но на практике такое случается редко по двум объективным причинам:

Часто данные формируются <b>несколькими независимыми процессами</b>, каждый из которых хранит данные в своей таблице.

<div style="background-color: #f5f5f5; padding: 15px; color: black; width: 80%;">Например, данные для отчёта по продажам могут состоять из списка банковских транзакций, курсов валют от Центробанка и планов отдела продаж из внутренней CRM. Все эти три таблицы, скорее всего, будут формироваться независимыми друг от друга системами. Объединять их в один отчёт придётся вам.</div>

Хранить все данные в одной таблице часто очень <b>накладно для ёмкости диска.</b>

<div style="background-color: #f5f5f5; padding: 15px; color: black; width: 80%;">Например, названия фильмов в наших данных хранятся в отдельной небольшой таблице. А в логах, которые могут растягиваться на многие миллионы строк, вместо названия фильма стоит его идентификатор. Числовой идентификатор фильма занимает на диске гораздо меньше места, чем длинное название, поэтому логи с идентификаторами будут занимать гораздо меньше места, чем единая таблица с названиями.</div>

<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;"> ✍ Прежде чем приступать к объединению таблиц, предлагаем вам исследовать информацию, которая в них содержится ↓</div>

### Задание 5.1

Значения из какого столбца таблиц ratings1 и ratings2 можно расшифровать с помощью таблицы movies?
- userId
- movieId
- title
- genres
- rating 
<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
 movieId
</code>
</details>

###  Задание 5.2

Сколько уникальных фильмов представлено в таблице movies?
<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
9742
</code>
</details>

In [1]:
import pandas as pd

In [2]:
movies_df = pd.read_csv("./data/movies.csv")

In [3]:
movies_df.nunique()

movieId    9742
title      9737
genres      951
dtype: int64

###   Задание 5.3

Сколько уникальных пользователей в таблице ratings1?
<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
274
</code>
</details>

In [5]:
ratings_df1 = pd.read_csv("./data/ratings1.csv")

In [6]:
ratings_df1.nunique()

userId      274
movieId    6219
rating       10
dtype: int64

###   Задание 5.4

В каком году было выставлено больше всего оценок?

Для ответа на этот вопрос используйте таблицу dates.
<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
2000
</code>
</details>

In [7]:
dates_df = pd.read_csv("./data/dates.csv")

In [11]:
dates_df["date"] = pd.to_datetime(dates_df["date"])

In [13]:
dates_df["year"] = dates_df["date"].dt.year

In [14]:
dates_df["year"].value_counts()

year
2000    10061
2017     8198
2007     7114
2016     6703
2015     6616
2018     6418
1996     6040
2005     5813
2012     4656
2008     4351
2009     4158
2006     4059
2003     4014
2001     3922
2002     3478
2004     3279
1999     2439
2010     2301
1997     1916
2011     1690
2013     1664
2014     1439
1998      507
Name: count, dtype: int64

Следуя нашему плану объединения таблиц, первым делом мы должны склеить таблицы ratings1 и ratings2 по строкам.

Для этого воспользуемся встроенной функцией Pandas <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html">concat()</a>, которая позволяет склеивать (конкатенировать) таблицы как по строкам, так и по столбцам.

### Основные параметры функции concat()

- objs — список объектов DataFrame ([df1, df2,…]), которые должны быть сконкатенированы;
- axis — ось определяет направление конкатенации: 0 — конкатенация по строкам (по умолчанию), 1 — конкатенация по столбцам;
- join — либо inner (пересечение), либо outer (объединение); рассмотрим этот момент немного позже;
- ignore_index — по умолчанию установлено значение False, которое позволяет значениям индекса оставаться такими, какими они были в исходных данных. Если установлено значение True, параметр будет игнорировать исходные значения и повторно назначать значения индекса в последовательном порядке.



<div style="background-color: #e0ffd1;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">Для корректной конкатенации по строкам объединяемые таблицы должны иметь одинаковую структуру — идентичное число и имена столбцов.</div>

Итак, давайте склеим  ratings1 и ratings2 по строкам, так как они имеют одинаковую структуру столбцов. Для этого передадим их списком в функцию concat(). Помним, что параметр axis по умолчанию равен 0, объединение происходит по строкам, поэтому не трогаем его. 

<div style="background-color: #e0ffd1;color: black;border: 3px solid black; padding: 15px; margin-right: 500px; width: 80%;">Примечание. Обратите внимание, что concat является функцией библиотеки, а не методом DataFrame. Поэтому её вызов осуществляется как pd.concat(...).</div>

In [15]:
ratings_df2 = pd.read_csv("./data/ratings2.csv")

In [16]:
ratings = pd.concat([ratings_df1, ratings_df2])

In [17]:
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
60831,610,166534,4.0
60832,610,168248,5.0
60833,610,168250,5.0
60834,610,168252,5.0


В результате мы увеличили первую таблицу, добавив снизу строки второй таблицы.

<div style="background-color: #e0fff3; padding: 15px; color: black; width: 80%;"> На первый взгляд может показаться, что всё прошло успешно, однако если мы посмотрим на индексы последних строк таблицы, то увидим, что их нумерация не совпадает с количеством строк. Это может привести к некорректному объединению таблиц по ключевым столбцам на следующем этапе решения нашей задачи.</div>

Это связано с тем, что по умолчанию concat сохраняет первоначальные индексы объединяемых таблиц, а обе наши таблицы индексировались, начиная от 0. Чтобы создать новые индексы, нужно выставить параметр ignore_index на True:

In [18]:
ratings = pd.concat([ratings_df1, ratings_df2], ignore_index=True)
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


Казалось бы, совсем другое дело! Но это ещё не всё. Давайте узнаем количество строк в таблицах ratings и dates, ведь нам предстоит вертикально склеить их между собой:

In [19]:
print("Число строк в таблице ratings:", ratings.shape[0])
print("Число строк в таблице dates", dates_df.shape[0])

Число строк в таблице ratings: 100837
Число строк в таблице dates 100836


Размерность таблиц разная — как такое могло произойти?

На самом деле очень просто: при выгрузке данных информация об оценках какого-то  пользователя попала в обе таблицы (ratings1 и ratings2). В результате конкатенации случилось дублирование строк. В данном примере их легко найти — выведем последнюю строку таблицы ratings1 и первую строку таблицы ratings2:

In [24]:
ratings_df1.tail(1)

,userId,movieId,rating
40000,274,5621,2.0


In [26]:
ratings_df2.head(1)

,userId,movieId,rating
0,274,5621,2.0


Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html">drop_duplicates()</a>, который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей, выставив параметр ignore_index в методе drop_duplicates() на значение True:

In [28]:
ratings = ratings.drop_duplicates(ignore_index=True)
print("Число строк в таблице ratings:", ratings.shape[0])

Число строк в таблице ratings: 100836


Наконец, мы можем добавить к нашей таблице с оценками даты их выставления. Для этого конкатенируем таблицы ratings и dates по столбцам:

In [29]:
ratings_dates = pd.concat([ratings, dates_df], axis=1)
ratings_dates

,userId,movieId,rating,date,year
0,1,1,4.0,2000-07-30 18:45:03,2000
1,1,3,4.0,2000-07-30 18:20:47,2000
2,1,6,4.0,2000-07-30 18:37:04,2000
3,1,47,5.0,2000-07-30 19:03:35,2000
4,1,50,5.0,2000-07-30 18:48:51,2000
...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,2017
100832,610,168248,5.0,2017-05-03 22:21:31,2017
100833,610,168250,5.0,2017-05-08 19:50:47,2017
100834,610,168252,5.0,2017-05-03 21:19:12,2017


<div style="border: 1px solid white; padding: 5px; margin-right: auto;  width: 80%;"> 
✍ Итак, мы смогли создать единую таблицу с рейтингами и датами их представления. Нашим следующим шагом будет присоединить к таблице информацию о фильмах из таблицы movies.

А пока предлагаем вам потренироваться в использовании функции concat() ↓
</div>

###  Задание 6.1

Какой параметр функции concat позволяет управлять способом конкатенации (проводить конкатенацию по строкам или по столбцам)?
- axis
- join
- keys
- ignore_index 

<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
 axis
</code>
</details>

###   Задание 6.2

Заданы две таблицы — df1 и df2. В первой содержатся имена и фамилии сотрудников, во второй — их должности.

df1 = pd.DataFrame({"Name": ["Pankaj", "Lisa"], "Surname": ["Sobolev", "Krasnova"]}) <br>
df2 = pd.DataFrame({"Role": ["Admin", "Editor"]})

Какой из приведённых ниже способов будет верным при объединении этих таблиц?

- df = pd.concat([df1, df2])

- df = pd.concat([df1, df2], axis=1)

- df = pd.concat([df1, df2], ignore_index=True)

- df = pd.concat([df2, df1])



<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
 df = pd.concat([df1, df2], axis=1)

</code>
</details>

###   Задание 6.3
В ваше распоряжение предоставлена директория users. В данной директории содержатся csv-файлы, в каждом из которых хранится информация об идентификаторах пользователей (user_id) и ссылки на их фотографии (image_url). Файлов в директории может быть сколько угодно.

Вам необходимо написать функцию concat_user_files(path), параметром которой является path — путь до директории. Функция должна объединить информацию из предоставленных вам файлов в один DataFrame и вернуть его.

Список названий всех файлов, находящихся в директории, вы можете получить с помощью функции os.listdir(path) из модуля os. Отсортируйте полученный список, прежде чем производить объединение файлов.

Обратите внимание, что метод os.listdir() возвращает только названия файлов в указанной директории, а при чтении файла необходимо указывать полный путь до него.

Не забудьте обновить индексы результирующей таблицы после объединения.

Примечание. Учтите, что на тестовом наборе файлов в результате объединения могут возникнуть дубликаты, от которых необходимо будет избавиться.

Например, для директории users/ результирующая таблица должна иметь следующий вид:<br>

<img src="../static/img/pandas_adv_6.png">



<details>
<summary><strong>Show answer</strong> (Click Here)</summary>
    &emsp; &emsp; <code>
 df = pd.concat([df1, df2], axis=1)

</code>
</details>

In [30]:
import os
import pandas as pd


def concat_user_files(path):
    files = os.listdir(path)
    full_df = pd.DataFrame()
    for file in files:
        df = pd.read_csv(os.path.join(path, file))
        full_df = pd.concat([full_df, df], ignore_index=True).drop_duplicates(
            ignore_index=True
        )
        full_df = full_df.sort_values(
            by="user_id", key=lambda x: x.str.slice(4).astype(int)
        ).reset_index(drop=True)
    return full_df

Имеется неточность в описании задания. Вместо директории users была использована директория users2. Ниже представлен код для поиска нужной директории:
<img src="../static/img/pandas_adv_7.jpg">